In [1]:
from utils import Settings
from openai import OpenAI
import json

api_key = Settings().openai_api_key
client = OpenAI(api_key=api_key)

In [ ]:
def fake_function(name, department):
  print(f"User {name}, department: {department}")

  data = {
      "name": name,
      "department": department,
      "date_of_birth": "1973-01-01"
  }

  return json.dumps(data) # zamiana typu python na json - serializacja

In [4]:
fake_function('adam', 'hr')

User adam, department: hr


'{"name": "adam", "department": "hr", "date_of_birth": "1973-01-01"}'

In [5]:
functions = [
    {
        "type": "function",
        "function": {
            "name": "find_user_data",
            "description": "Retrieves user's data form the database by a given name and a department name",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "The name of the user"
                    },
                    "department": {
                        "type": "string",
                        "description": "The department's name",
                        "enum": ["IT", "HR", "LEGAL"]
                    }
                },
                "required": ["name", "department"]
            }
        }
    }
]

In [6]:
system_messages = [
    {"role": "system", "content": "You are a HR helper who makes database queries"},
    {"role": "system", "content": "You can retrieve employee's data from the database by the person's name and the department (IT, HR, LEGAL)."},
    {"role": "system", "content": "Current date: 2025-11-06"}
]

In [7]:
messages = system_messages + [{"role": "user", "content": "How old is John from the legal department?"}]

In [8]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=functions
)

In [20]:
for tool_call in response.choices[0].message.tool_calls:

    function_args = json.loads(tool_call.function.arguments)
    function_name = tool_call.function.name

    if function_name == "find_user_data":
        result = fake_function(**function_args)
        print(result)

User John, department: LEGAL
{"name": "John", "department": "LEGAL", "date_of_birth": "1973-01-01"}


In [25]:
response_msg = {
    "role": 'tool',
    "name": function_name,
    'content': result,
    "tool_call_id": response.choices[0].message.tool_calls[0].id
}

In [ ]:
messages.append(response.choices[0].message) # dodanie do historii ze model prosił o wywołanie tej funkcji
messages.append(response_msg) # dodanie do historii wyników działania tej funkcji

In [29]:
messages

[{'role': 'system',
  'content': 'You are a HR helper who makes database queries'},
 {'role': 'system',
  'content': "You can retrieve employee's data from the database by the person's name and the department (IT, HR, LEGAL)."},
 {'role': 'system', 'content': 'Current date: 2025-11-06'},
 {'role': 'user', 'content': 'How old is John from the legal department?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_2wGRs0ILAUnEN4nBDVz8aU5t', function=Function(arguments='{"name":"John","department":"LEGAL"}', name='find_user_data'), type='function')]),
 {'role': 'tool',
  'name': 'find_user_data',
  'content': '{"name": "John", "department": "LEGAL", "date_of_birth": "1973-01-01"}',
  'tool_call_id': 'call_2wGRs0ILAUnEN4nBDVz8aU5t'}]

In [30]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages, # pełna historia konwersjacji
    tools=functions
)

In [31]:
response

ChatCompletion(id='chatcmpl-CYvkAaNfHG5leZbkv9XB7pQVtWYfj', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='John from the legal department was born on January 1, 1973, which makes him 52 years old as of now.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1762441154, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_a788c5aef0', usage=CompletionUsage(completion_tokens=28, prompt_tokens=192, total_tokens=220, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))